In [ ]:
# System packages
import os
import re
import warnings
from datetime import tzinfo, timedelta, datetime, date

# Data storage and management
import pandas as pd
import numpy as np

# Plotting packages
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import matplotlib.ticker as ticker
import seaborn as sns

# Plotnine -- ggplot analog
import plotnine
from plotnine import *

In [ ]:
#Set up the google bucket
import os
import subprocess
import numpy as np
import pandas as pd




In [ ]:
def getFromBucket(name_of_file_in_bucket):
    # get the bucket name
    my_bucket = os.getenv('WORKSPACE_BUCKET')

    # copy csv file from the bucket to the current working space
    os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

    print(f'[INFO] {name_of_file_in_bucket} is successfully downloaded into your working space')
    # save dataframe in a csv file in the same workspace as the notebook
    my_dataframe = pd.read_csv(name_of_file_in_bucket)
    return my_dataframe

In [ ]:
def storeInBucket(name_of_file_loc):
    # get the bucket name
    
    # This snippet assumes you run setup first

    # This code saves your dataframe into a csv file in a "data" folder in Google Bucket

    # Replace df with THE NAME OF YOUR DATAFRAME
    my_dataframe = pd.read_csv(name_of_file_loc)   

    # Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
    destination_filename = name_of_file_loc

    ########################################################################
    ##
    ################# DON'T CHANGE FROM HERE ###############################
    ##
    ########################################################################

    # save dataframe in a csv file in the same workspace as the notebook
    my_dataframe.to_csv(destination_filename, index=False)

    # get the bucket name
    my_bucket = os.getenv('WORKSPACE_BUCKET')

    # copy csv file to the bucket
    args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
    output = subprocess.run(args, capture_output=True)

    # print output from gsutil
    output.stderr
    

Set up data

In [ ]:
# Get a representation of the Workspace's connection to the AoU Curated Data Repository
WORKSPACE_CDR = os.getenv('WORKSPACE_CDR')
# Get the associated GOOGLE_PROJECT billing
MY_BILLING = os.getenv("GOOGLE_PROJECT")

In [ ]:
# Date of first COVID-19 case reported in the United States
FIRST_COVID_DX_DATE_IN_US = "2020-01-20 00:00:00"

# Concept IDs for "Positive" or "Detected"
CONCEPT_IDS_POS = [9191, 4126681, 45877985, 45884084, 36032716, 36715206]


# Concept IDs for qualifying COVID-19 condition codes (ICD-10-CM U07.1 or OMOP COVID concept)
COVID_CONDITION_LIST = [702953, 37311061]

Let's find all the different types of COVID-related laboratory tests available in AoU dataset by searching the MEASUREMENT table for lab tests with different variations on 'SARS-CoV-2' or 'COVID'.

There are a few records for tests that appear to be combined SARS-CoV-2 and Influenza PCRs. Without a clear way to differentiate what a 'Positive' result means for those tests, we will exclude those records.

In [ ]:
# Distribution of COVID related measurements and their results in the dataset
query = f"""
    SELECT
        concept.concept_name
        , concept.concept_id
        , concept.vocabulary_id
        , concept.concept_class_id
        , concept.concept_code
        , m.value_source_value
        , m.value_as_concept_id
        , result_concepts.concept_name AS result
        , COUNT(DISTINCT measurement_id) AS num_measurements
        , COUNT(DISTINCT person_id) AS num_person
    FROM
        {WORKSPACE_CDR}.`measurement` AS m
    INNER JOIN
        {WORKSPACE_CDR}.`concept` AS concept
        ON concept.concept_id = m.measurement_concept_id
    LEFT JOIN
        {WORKSPACE_CDR}.`concept` AS result_concepts
        ON result_concepts.concept_id = m.value_as_concept_id
    WHERE
        /* 'SARS-CoV-2' or 'COVID' related measurements */
        (
            LOWER(concept.concept_name) like '%sars-cov-2%'
            OR LOWER(concept.concept_name) like '%sars-cov2%'
            OR LOWER(concept.concept_name) like '%sarscov-2%'
            OR LOWER(concept.concept_name) like '%sarscov2%'
            OR LOWER(concept.concept_name) like '%covid%'
        )
    AND /* Not a combined SARS-CoV-2 & Influenza panel */
        (
            LOWER(concept.concept_name) not like '%influenz%'
        )
    AND /* Test type of interest */
        (
            /* Nucleic acid amplification tests (NAAT) and other PCR / nucleic acid tests */
            LOWER(concept.concept_name) like '%naa%'
            OR LOWER(concept.concept_name) like '%rna%'
            OR LOWER(concept.concept_name) like '%dna%'
            OR LOWER(concept.concept_name) like '%nucleic acid%'

            /*  Can also include antigent tests for sensitivity analysis, but sample size is smaller */
            OR LOWER(concept.concept_name) like '%antigen%'
            OR LOWER(concept.concept_name) like '% ag %'     
        )
    GROUP BY 
        concept.concept_name
        , concept.concept_id
        , concept.vocabulary_id
        , concept.concept_class_id
        , concept.concept_code
        , m.value_source_value
        , m.value_as_concept_id
        , result
"""

# Query database and download data
df = pd.read_gbq(query, dialect="standard")

We will then get the CONCEPT_IDs for these SARS-CoV-2 tests using the data we pulled from the MEASUREMENT table.

In [ ]:
# Get the list of the CO SARS-CoV-2 tests in measurement table: 
COVID_TEST_CONCEPT_IDS = df.loc[:, 'concept_id'].unique()
# Alternative hard-coded list (for reproducibility)
# COVID_TEST_CONCEPT_IDS = [706163, 706170, 700360, 706169, 706160, 704059, 706158, 723478, 706161, 586524, 723476, 586526, 706173, 715272, 742219]

Help function for plotting number of participants positive for COVID-19

In [ ]:
# Help function for plotting number of participants positive for COVID-19
def plot_positive_result_histogram(plt_data, plot_title_text="", x_var="event_datetime", xlab="", ylab="",
                                   date_limits = [date.fromisoformat('2020-02-01'), date.fromisoformat('2022-02-15')],
                                   label_data_string=None, date_breaks="2 months"):
    # Set up plot features
    bar_color = "white"

    # X-axis text
    axis_text_x_size = 12
    axis_text_x_hjust = 1
    axis_text_x_angle = 30
    axis_text_x = element_text(size=axis_text_x_size, angle=axis_text_x_angle, hjust=axis_text_x_hjust)

    # Y-axis text
    axis_text_y_size = axis_text_x_size
    axis_text_y = element_text(size=axis_text_y_size)

    # Plot title text
    plot_title_size = 16
    plot_title = element_text(size=plot_title_size)

    # 2-week bins on x-axis
    x_bins = pd.date_range(start=date_limits[0], end=date_limits[1], freq='SMS')

    # Expand the x-axis by 14 units on the upper limit
    x_expand = (0, 0, 0, 14)
    # Expand the y-axis by 25% multiplication on the upper limit
    y_expand = (0, 0, 0.25, 0)

    # Data for labels
    # If label is missing, use the number of unique person_id in plot data
    if label_data_string is None:
        label_data_string = f"N: {plt_data.loc[:, 'person_id'].unique().shape[0]:,}"
    # Figure out what is the largest histogram bin height using the numpy.histogram function
    hist, bin_edges = np.histogram(a=plt_data.loc[:, x_var].values.astype(np.int64), bins=x_bins.values.astype(np.int64))
    # Use the highest histogram height, rounded up to nearest 200
    label_y = np.ceil(np.max(hist) / 200) * 200
    label_data = pd.DataFrame(data={'x':x_bins[2], 'y':label_y, 'label':label_data_string}, index=[0])
    label_size = 10
    label_pt = label_size

    # Save plot
    plot = ggplot(data=plt_data, mapping=aes(x=x_var)) + \
        geom_histogram(color=bar_color, breaks=x_bins) + \
        scale_x_datetime(date_labels="%b-%Y", date_breaks = date_breaks, limits=date_limits, expand=x_expand) + \
        scale_y_continuous(expand = y_expand) + \
        labs(x=xlab, y=ylab, title=plot_title_text) + \
        theme_bw() + \
        theme(axis_text_x = axis_text_x
              , axis_text_y = axis_text_y
              , plot_title = plot_title)
    
    plot = plot + geom_label(mapping=aes(x='x', y='y', label='label'), data=label_data, ha='left')
    # Return plot
    return(plot)

# Data retrieval
Retreive records for SARS-CoV-2 cases

In [ ]:
query = f"""
/* Demographics data */
    SELECT
        person.person_id
        , person.gender_concept_id
        , p_gender_concept.concept_name as gender
        , person.birth_datetime as date_of_birth
        , person.race_concept_id
        , p_race_concept.concept_name as race
        , person.ethnicity_concept_id
        , p_ethnicity_concept.concept_name as ethnicity
        , person.sex_at_birth_concept_id
        , p_sex_at_birth_concept.concept_name as sex_at_birth 
        , event_datetime
        , event_type
        , event_concept_id
        , event_name
        , 'TRUE' as covid_positive
    FROM
        {WORKSPACE_CDR}.`person` person 
    LEFT JOIN
        {WORKSPACE_CDR}.`concept` p_gender_concept 
        ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        {WORKSPACE_CDR}.`concept` p_race_concept 
        ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        {WORKSPACE_CDR}.`concept` p_ethnicity_concept 
        ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        {WORKSPACE_CDR}.`concept` p_sex_at_birth_concept 
        ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    INNER JOIN (
        /* COVID-positive by a lab test result (measurement table) */
        SELECT
                m.person_id as person_id
                , m.measurement_datetime as event_datetime
                , 'measurement' as event_type
                , m.measurement_concept_id as event_concept_id
                , m_concept.concept_name as event_name
        FROM
            {WORKSPACE_CDR}.`measurement` AS m
        INNER JOIN
            {WORKSPACE_CDR}.`concept` AS m_concept
            ON m_concept.concept_id = m.measurement_concept_id
        LEFT JOIN
            {WORKSPACE_CDR}.`concept` AS result_concepts
            ON result_concepts.concept_id = m.value_as_concept_id
        WHERE
        /* SARS-CoV-2 lab tests tests */
            (
                /* m.measurement_concept_id in (706163,706170,700360,706169,706160,704059,723476,706158,757685,723478,723477,706161,586524,586526,715261,742224,742218,715272,36032419,706173,757677,723464)
                */
                m.measurement_concept_id in ({','.join([str(x) for x in COVID_TEST_CONCEPT_IDS])})
            )
        AND 
        /* Test result is 'Positive' or 'Detected' */
            (
                /* m.value_as_concept_id in (9191,4126681,45877985,45884084,36032716,36715206)
                */
                m.value_as_concept_id in ({','.join([str(x) for x in CONCEPT_IDS_POS])})

            )
        
        AND 
        /* Ensure date of the test is plausible: i.e. after date of first case in the United States */
        /*  m.measurement_datetime >= TIMESTAMP('2020-01-20 00:00:00') */
            m.measurement_datetime >= TIMESTAMP('{FIRST_COVID_DX_DATE_IN_US}')
    UNION ALL
        /* COVID-positive by a diagnosis code (condition_occurrence table) */
        SELECT
            co.person_id as person_id
            , co.condition_start_datetime as event_datetime
            , 'condition' as event_type
            , co.condition_concept_id as event_concept_id
            , co_concept.concept_name as event_name
        FROM 
             {WORKSPACE_CDR}.`condition_occurrence` AS co
        INNER JOIN
            {WORKSPACE_CDR}.`concept` AS co_concept
            ON co_concept.concept_id = co.condition_concept_id
        WHERE
        /* Presence of relevant COVID-19 diagnoses code either as condition_source_concept_id or condition_concept_id */
            (
                co.condition_source_concept_id IN (702953, 37311061)
                OR co.condition_concept_id IN (702953, 37311061)
            )
         AND 
         /* Ensure date of the diagnosis code is plausible: i.e. after date of first case in the United States */
         /* co.condition_start_datetime >= TIMESTAMP('2020-01-20 00:00:00') */
            co.condition_start_datetime >= TIMESTAMP('{FIRST_COVID_DX_DATE_IN_US}')
    UNION ALL
        /* Self-reported COVID-positive by COPE survey (ds_survey table) */
        SELECT
            answer.person_id as person_id
            , answer.survey_datetime as event_datetime
            , 'survey' as event_type
            , answer.survey_version_concept_id as event_concept_id
            , answer.survey_version_name as event_name
        FROM
             {WORKSPACE_CDR}.`ds_survey` answer   
        WHERE
            /* Validation check to ensure the question_concept_id is a valid survey-related concept_id */
            question_concept_id IN (
                SELECT
                    DISTINCT(question_concept_id) as concept_id  
                FROM
                     {WORKSPACE_CDR}.`ds_survey` 
                )
        AND 
            /* question_concept_id for 'Was the test for COVID-19 positive?' */
            question_concept_id IN (1333326)
        AND
            /* answer_concept_id indicating test result was 'Yes' or 'Yes,some' */
            answer_concept_id IN (1332898, 1310131) 
    ) AS covid_positive
    USING(person_id)
    ;
"""

# Query database and download data
df = pd.read_gbq(query, dialect="standard")

# Column names for database pull results
covid_pos = 'covid_pos'
person_id = 'person_id'
event_type = 'event_type'
event_datetime = 'event_datetime'
event_name = 'event_name'
covid_positive = 'covid_positive'

In [ ]:
# Display number of participants
print(f"Number of participants with \u2265 1 positive SARS-CoV-2 laboratory test: {df.loc[:, person_id].unique().shape[0]:,}")

Distribution of COVID-19 positive participants in AoU over time by ascertainment method

In [ ]:
figsize = [8, 2]
for group in df.loc[:, event_type].unique():
    plt_data = df.loc[df[event_type] == group, [person_id, event_datetime]].groupby([person_id]).agg(np.min).reset_index()
    label_data_string = f"AoU participants COVID-19 positive by {group} data: {plt_data.shape[0]:,}"
    tmp = plot_positive_result_histogram(plt_data=plt_data, label_data_string=label_data_string)
    tmp = tmp + theme(figure_size = figsize)
    print(tmp)
    del plt_data, label_data_string, tmp

plt_data = df.loc[:, [person_id, event_datetime]].groupby([person_id]).agg(np.min).reset_index()
label_data_string = f"AoU participants COVID-19 positive by any method: {plt_data.shape[0]:,}"
plot = plot_positive_result_histogram(plt_data=plt_data, label_data_string=label_data_string)
plot = plot + theme(figure_size = figsize)
print(plot)
del plot, plt_data, label_data_string

PPI Survey and COPE Survey data

In [ ]:
#This cell takes like 5 minutes to run btw
query = f"""
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
         {WORKSPACE_CDR}.`ds_survey` answer   
    WHERE
        (
            /* Validation check to ensure the question_concept_id is a valid survey-related concept_id */
            question_concept_id IN (
                SELECT
                    DISTINCT(question_concept_id) as concept_id  
                FROM
                     {WORKSPACE_CDR}.`ds_survey` 
            )
        )  
            /* Participants in the AoU COVID-19 cohort */
    AND (
            answer.person_id IN ({','.join(df.loc[:, person_id].unique().astype(str))})
        )
        ;
    """
# Query database and download data
survey_data = pd.read_gbq(query, dialect="standard")

# Column names in survey data
question = 'question'
question_concept_id = 'question_concept_id'
answer_concept_id = 'answer_concept_id'
answer = 'answer'
survey = 'survey'
survey_datetime = 'survey_datetime'
survey_version_concept_id = 'survey_version_concept_id'
survey_version_name = 'survey_version_name'

# Add a "Default" values for survey_version_name, survey_version_concept_id
survey_data.fillna(value={survey_version_name: 'PPI', survey_version_concept_id: 0}, inplace=True, downcast='infer')

Response rates for PPI survey questions

In [ ]:
# Count number of distinct Participants
COUNT_COL = person_id
# For each question, across each survey type
GROUP_COLS = [survey, survey_version_name, survey_version_concept_id, question, question_concept_id]
NUNIQUE = 'nunique'
PROPORTION = 'proportion'
# Unique column name suffixes for joins
MERGE_SUFFIXES=["_total", "_skip", '_survey']

# Number of participants who responded to each survey
survey_response = survey_data.loc[:, [person_id, survey_version_name, survey_version_concept_id]
               ].groupby([survey_version_name, survey_version_concept_id]
                ).agg(
                    **{NUNIQUE+MERGE_SUFFIXES[2]: pd.NamedAgg(column=COUNT_COL, aggfunc=NUNIQUE)}
                )

# Distinct participants for each question
survey_counts = survey_data.loc[:, [COUNT_COL] + GROUP_COLS].groupby(GROUP_COLS, dropna=False
                          ).agg(
                                # This dictionary unpacking (**kwards) sends 'nunique' as a keyword and pd.NamedAgg(...) as a value to 'agg' method
                                **{NUNIQUE: pd.NamedAgg(column=COUNT_COL, aggfunc=NUNIQUE)}
                                # {COUNT_COL: "nunique"}
                               ).sort_values(NUNIQUE, ascending=False)

# Number of participants who's answer was "Skip" for each question
survey_counts = survey_counts.join(survey_data.loc[survey_data[answer].str.contains("PMI: Skip")
                                     , [COUNT_COL] + GROUP_COLS].groupby(GROUP_COLS, dropna=False
                            ).agg(
                                # This dictionary unpacking (**kwards) sends 'nunique' as a keyword and pd.NamedAgg(...) as a value to 'agg' method
                                **{NUNIQUE: pd.NamedAgg(column=COUNT_COL, aggfunc=NUNIQUE)}
                                # {COUNT_COL: "nunique"}
                            ).sort_values(NUNIQUE, ascending=False
                            )
                  , how='left', lsuffix=MERGE_SUFFIXES[0], rsuffix=MERGE_SUFFIXES[1])


# Add in number of participants who responded to each survey
survey_counts = survey_counts.join(survey_response)

# Set 'nunique_skip' to pd.Int64Dtype() to allow 'missing' integer value
survey_counts[NUNIQUE+MERGE_SUFFIXES[1]] = survey_counts[NUNIQUE+MERGE_SUFFIXES[1]].astype(pd.Int64Dtype())

# Calculate proportion of COVID-19 positive participants who answered each question 
survey_counts[PROPORTION+MERGE_SUFFIXES[0]] = survey_counts[NUNIQUE+MERGE_SUFFIXES[0]] / survey_data[person_id].unique().shape[0]

# Calculate proportion of COVID-19 positive participants whose answer for each question was "PMI: Skip"
survey_counts[PROPORTION+MERGE_SUFFIXES[1]] = survey_counts[NUNIQUE+MERGE_SUFFIXES[1]] / survey_counts[NUNIQUE+MERGE_SUFFIXES[0]]

# Calculate response rate for each question versus total number of participants for each survey
survey_counts[PROPORTION+MERGE_SUFFIXES[2]] = survey_counts[NUNIQUE+MERGE_SUFFIXES[0]] / survey_counts[NUNIQUE+MERGE_SUFFIXES[2]]


PPI survey questions with high response rates

In [ ]:
# Count number of distinct Participants
COUNT_COL = person_id
# For each question, across each survey type
GROUP_COLS = [survey, survey_version_name, survey_version_concept_id, question, question_concept_id]


# Set of key, value pairs to sort the survey questions
survey_key_levels = {'The Basics': 0, "Lifestyle": 1, 'Overall Health': 2, 'Family History': 3
                    , 'Personal Medical History': 4, 'Healthcare Access & Utilization': 5
                    , 'Social Determinants of Health': 6}

# Response rate cutoff
RESPONSE_CUTOFF = 0.80
# Get the question_concept_ids with good response rates, sorted by what part of the survey they are in
question_ids_high_response_rate = \
    survey_counts.loc[pd.IndexSlice[:, 0, :, :, survey_counts[PROPORTION+"_survey"].ge(RESPONSE_CUTOFF)]
                      , :].index.to_frame(index=False
                                     ).sort_values(by=question_concept_id, ascending=False
                                     ).sort_values(by='survey', key=lambda x: x.map(survey_key_levels)
                                     )[question_concept_id]

# Show the questions with high resonse rates
survey_counts.loc[pd.IndexSlice[:, 0, :, :, survey_counts[PROPORTION+"_survey"].ge(RESPONSE_CUTOFF)]
                                , :
                 ].reset_index().loc[:, [survey, question, question_concept_id] + [NUNIQUE+s for s in MERGE_SUFFIXES[0:2]] + [PROPORTION+s for s in MERGE_SUFFIXES]
                                    ].sort_values(by=survey, key=lambda x: x.map(survey_key_levels)
                                                 )

In [ ]:
survey_counts.loc[pd.IndexSlice[:, 0, :, :, survey_counts[PROPORTION+"_survey"].ge(RESPONSE_CUTOFF)]
                                , :
                 ].reset_index().loc[:, [survey, question, question_concept_id] + [NUNIQUE+s for s in MERGE_SUFFIXES[0:2]] + [PROPORTION+s for s in MERGE_SUFFIXES]
                                    ].sort_values(by=survey, key=lambda x: x.map(survey_key_levels)
                                                 )

In [ ]:
# Plot the distributions of answers for each PPI survey question with a high response rate
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    # Plot distribution of reponses for each question with high response rate
    for question_id in question_ids_high_response_rate:
        # Temporary plotting data -- participant counts per answer
        df_tmp = survey_data.loc[survey_data[question_concept_id] == question_id, [person_id, answer]
                                ].groupby([answer], dropna=False
                                ).agg(
                                **{NUNIQUE: pd.NamedAgg(column=COUNT_COL, aggfunc=NUNIQUE)}
                                ).reset_index()
        # Sort the count data answer text such that any answers with "PMI" are displayed last / lowest
        df_tmp.sort_values(by=answer, key=lambda x: x.str.contains('PMI'), inplace=True, ascending=False)
        # Set title of plot as question text 
        tmp_question = survey_data.loc[survey_data[question_concept_id] == question_id, question].unique()[0]
        # Include total number of responses in a text box
        tmp_box_text = f"Number of responses: {df_tmp[NUNIQUE].sum():,}"
        tmp_box_props = dict(boxstyle='round', facecolor='white', alpha=1)
        tmp_box_position={'x':0.5, 'y':0.10}
        # Labels of plot
        xlabel = 'Number of respondents'
        # Plot distribution of answers for each question
        ax = df_tmp.plot.barh(x=answer, y=NUNIQUE, title=tmp_question, legend=False, xlabel='')
        ax.text(**tmp_box_position, s=tmp_box_text, transform=ax.transAxes, fontsize=10, verticalalignment='top', bbox=tmp_box_props)
        ax.set_xlabel(xlabel)
        ax.set_ylabel('')

        del ax, df_tmp, tmp_question, tmp_box_text, tmp_box_props, tmp_box_position, xlabel

# PPI survery question encoding
Make binary scales for 'Yes'/'No' survey data

In [ ]:
# Dictionary to map 'yes'/'no' to 0 and 1
binary2map = {'yes': 1, 'no': 0}

# Add binary_score column for records with relevant answers by extracting the word after ":" in answers
BINARY_SCORE = 'binary_score'
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    survey_data[BINARY_SCORE] = survey_data.loc[survey_data[answer].str.contains('\w+: (yes|no)$', flags=re.I) &
                                            survey_data[question_concept_id].isin(question_ids_high_response_rate)
                                            , answer
                                           ].str.extract(pat='\w+:\s+(.+)', flags=re.I, expand=False
                                           ).str.lower().map(binary2map).astype(pd.Int64Dtype())

Convert income levels into an ordinal scale

In [ ]:
# Column names
idx = 'idx'
jdx = 'jdx'
INCOME_LEVEL = 'income_level'

# Possible income value answers
INCOME_QUESTION_CONCEPT_ID = 1585375
income_values = survey_data.loc[survey_data[question_concept_id].isin([INCOME_QUESTION_CONCEPT_ID])
                , answer]

# Extract income numbers and sort numerically. Replace 'less 10k' with '0k' and 'more 200k' with '200k'
tmp = income_values.str.replace(r'less', '0k', regex=True
     ).str.replace(r'more', '200k', regex=True
     ).str.extract(r'annual income: (?P<income_level>\d+k)', flags=re.I, expand=True
     ).assign(**{idx:lambda x: x[INCOME_LEVEL].str.extract('(\d+)k', expand=False).astype(float)}
     ).sort_values(by=idx, ascending=True
     )
# Use groupby.ngroup to count number of distinct 'income_level' groups, sorted numerically, to get an ordinal scale
tmp[jdx] = tmp.groupby(idx).ngroup() + 1
tmp.loc[tmp[INCOME_LEVEL].isna(), jdx] = np.nan
# Export ordinal scale for income_level to a dictionary mapper
income_mapper = tmp.loc[:, [INCOME_LEVEL, jdx]].drop_duplicates().set_index(INCOME_LEVEL).to_dict()[jdx]
del tmp

# Transform the income level into an ordinal scale
survey_data[INCOME_LEVEL] = survey_data.loc[survey_data[question_concept_id].isin([INCOME_QUESTION_CONCEPT_ID])
                                            , answer
                                           ].str.replace(r'less', '0k', regex=True
                                           ).str.replace(r'more', '200k', regex=True
                                           ).str.extract(r'annual income: (?P<income_level>\d+k)', flags=re.I, expand=False
                                           ).map(income_mapper)

Convert education levels into an ordinal scale

In [ ]:
# Ordinal scale education level mapper
education7map = {
    'Never Attended':1
    ,'One Through Four':2
    ,'Five Through Eight':3
    ,'Nine Through Eleven':4
    ,'Twelve Or GED':5
    ,'College One to Three':6
    ,'College Graduate':7
    ,'Advanced Degree':8
    , np.nan: 0
}

# Column name
EDUCATION_LEVEL = 'education_level'
EDUCATION_QUESTION_CONCEPT_ID = 1585940

# Transform the education level into an ordinal scale
survey_data[EDUCATION_LEVEL] = survey_data.loc[survey_data[question_concept_id].isin([EDUCATION_QUESTION_CONCEPT_ID])
                                               , answer
                                              ].str.extract(r'\w+: (?P<education_level>.+)', flags=re.I, expand=False
                                           ).map(education7map)

Convert levels of duration in current living situation into an ordinal scale

In [ ]:
idx = 'idx'

# Possible living duration value answers and question_concept_id
LIVING_DURATION_CONCEPT_ID = 1585879
LIVING_DURATION = 'living_duration'

# Extract living duration and sort numerically. Replace 'less' with '0' and '20' with '20'
tmp = survey_data.loc[survey_data[question_concept_id].isin([LIVING_DURATION_CONCEPT_ID])
                , answer
               ].str.replace(r'less', '0', regex=True
               ).str.replace(r'more', '20', regex=True
               ).str.extract(r'\w+: (?P<living_duration>\d+)', flags=re.I, expand=True
               ).astype(float).astype(pd.Int64Dtype()
               ).sort_values(by=LIVING_DURATION)

# Use groupby.ngroup to count number of distinct 'income_level' groups, sorted numerically, to get an ordinal scale
tmp[idx] = tmp.groupby(LIVING_DURATION).ngroup() + 1
tmp.loc[tmp[LIVING_DURATION].isna(), idx] = np.nan

# Export ordinal scale for living_duration to a dictionary mapper
living_duration_mapper = tmp.loc[:, [LIVING_DURATION, idx]].drop_duplicates().set_index(LIVING_DURATION).to_dict()[idx]
del tmp

# Transform the living duration into an ordinal scale
survey_data[LIVING_DURATION] = survey_data.loc[survey_data[question_concept_id].isin([LIVING_DURATION_CONCEPT_ID])
                                               , answer
                                              ].str.replace(r'less', '0', regex=True
                                              ).str.replace(r'more', '20', regex=True
                                              ).str.extract(r'\w+: (?P<living_duration>\d+)', flags=re.I, expand=False
                                              ).astype(float).astype(pd.Int64Dtype()
                                              ).map(living_duration_mapper)

Convert Likert Scales for 'Overall Health' questions from text to numerical scale

In [ ]:
# Dictionary to map various Likert Scales to similar 5-point ordinal scale
likert5map = {'excellent': 5, 'excllent':5, 'very good': 4, 'good': 3, 'fair': 2, 'poor': 1
        , 'always': 5, 'often': 4, 'sometimes': 3, 'occasionally': 2, 'rarely': 2, 'never': 1
        , 'extremely': 5, 'quite a bit': 4, 'somewhat': 3, 'a little bit': 2, 'not at all': 1
        , 'very severe': 5, 'severe': 4, 'moderate': 3, 'mild': 2, 'none': 1
        , 'completely': 5, 'mostly': 4, 'moderately': 3, 'a little': 2
       }

# Add likert_score column for records with relevant answers
LIKERT_SCORE = 'likert_score'
survey_data[LIKERT_SCORE] = survey_data.loc[survey_data[survey].eq('Overall Health')
                                            , answer
                                           ].str.extract(pat='\w+:\s+(.+)', flags=re.I, expand=False
                                           ).str.lower(
                                           ).map(likert5map
                                           ).astype(pd.Int64Dtype())

# Average pain  in last 7 days -- get numerical values from 0 to 10
AVG_PAIN_QUESTION_CONCEPT_ID=1585747
survey_data[LIKERT_SCORE] = survey_data.loc[survey_data[survey].eq('Overall Health') & \
                                            survey_data[question_concept_id].isin([AVG_PAIN_QUESTION_CONCEPT_ID])
                                            , answer
                                           ].str.extract(r'(\d+)', flags=re.I, expand=False
                                           ).astype(float
                                           ).astype(pd.Int64Dtype())

In [ ]:
survey_data.loc[survey_data[answer].str.contains('\w+: (yes|no)$', flags=re.I) &
                survey_data[question_concept_id].isin(question_ids_high_response_rate)
                , answer].value_counts().sort_index()

In [ ]:
COUNT_COL = person_id
GROUP_COLS = [question_concept_id, question, answer, answer_concept_id]
tmp = survey_data.loc[
    survey_data[survey].str.match('Overall Health')
    , :
    ].groupby(GROUP_COLS
    ).agg(
        **{NUNIQUE: pd.NamedAgg(column=COUNT_COL, aggfunc=NUNIQUE)}
    ).reset_index()

# Number of distinct answers per question
GROUP_COLS = [question, question_concept_id]
COUNT_COL = answer_concept_id
answer_counts = tmp.loc[:, [COUNT_COL] + GROUP_COLS].groupby(GROUP_COLS).agg(
                **{NUNIQUE: pd.NamedAgg(column=COUNT_COL, aggfunc=NUNIQUE)}
                )
answer_counts

#
COUNT_COL = person_id
GROUP_COLS = [question_concept_id, question, answer, answer_concept_id]
tmp = survey_data.loc[
    (survey_data[question_concept_id].isin(answer_counts.loc[answer_counts[NUNIQUE].eq(6), :].index.get_level_values(1)) | \
     survey_data[question_concept_id].eq(1585729)) & \
    survey_data[question_concept_id].isin(question_ids_high_response_rate)
    , :
    ].groupby(GROUP_COLS
             ).agg(
        **{NUNIQUE: pd.NamedAgg(column=COUNT_COL, aggfunc=NUNIQUE)}
    ).reset_index()

LIKERT_SCORE = 'likert_score'
tmp[LIKERT_SCORE] = tmp[answer].str.extract(pat='\w+:\s+(.+)', flags=re.I, expand=False).str.lower().map(likert5map).astype(pd.Int64Dtype())

In [ ]:
# smoking_questions =  survey_data.loc[survey_data[question].str.contains('smoking', flags=re.I), [question, question_id]].drop_duplicates()
# smoking_questions
tobacco_questions =  survey_counts.index.get_level_values(3)[
                        survey_counts.index.get_level_values(3).to_series().str.match('Smoking:') 
                        ].values
tobacco_question_concept_ids = survey_counts.loc[pd.IndexSlice[:, 0, :, tobacco_questions, :], :].index.get_level_values(4).values
                   

survey_data.loc[
    survey_data[person_id].isin(
        survey_data.loc[
            survey_data[question_concept_id].isin([1585857]) & \
            survey_data[answer_concept_id].isin([1585858])
            , person_id
            ]
        ) & \
    survey_data[question_concept_id].isin(tobacco_question_concept_ids)
 , :
].groupby(GROUP_COLS
                ).agg(
                **{NUNIQUE: pd.NamedAgg(column=COUNT_COL, aggfunc=NUNIQUE)}
                )

In [ ]:
# survey_data.loc[survey_data[answer_concept_id].isna(), :]
survey_data.loc[survey_data[question_concept_id].eq(1586159), :]

In [ ]:
1585864

query = f"""
    SELECT * 
    FROM
         {WORKSPACE_CDR}.`ds_survey` answer   
    LIMIT 1000
        ;
    """
# Query database and download data
data = pd.read_gbq(query, dialect="standard")

In [ ]:
import pandas
import os

# This query represents dataset "TESTING_COVID_DATASET" for domain "person" and was generated for All of Us Controlled Tier Dataset v6
dataset_34296773_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (702953) 
                                AND is_standard = 0 
                            )) criteria 
                    UNION
                    ALL SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (706163) 
                                AND is_standard = 1  
                                AND  value_as_concept_id IN (9191, 4126681, 45877985, 45884084)
                            )) criteria 
                    UNION
                    ALL SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (706170) 
                                AND is_standard = 1  
                                AND  value_as_concept_id IN (45877985, 45884084)
                            )) criteria 
                    UNION
                    ALL SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (700360) 
                                AND is_standard = 1  
                                AND  value_as_concept_id IN (45884084, 45877985)
                            )) criteria 
                    UNION
                    ALL SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (706169) 
                                AND is_standard = 1  
                                AND  value_as_concept_id IN (45877985, 45884084)
                            )) criteria ) 
                )"""

dataset_34296773_person_df = pandas.read_gbq(
    dataset_34296773_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_34296773_person_df.head(5)

In [ ]:
# This query represents dataset "TESTING_COVID_DATASET" for domain "survey" and was generated for All of Us Controlled Tier Dataset v6
dataset_34296773_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (
                SELECT
                    DISTINCT(question_concept_id) as concept_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` 
            )
        )  
        AND (
            answer.PERSON_ID IN (
                SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (
                        SELECT
                            criteria.person_id 
                        FROM
                            (SELECT
                                DISTINCT person_id,
                                entry_date,
                                concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                            WHERE
                                (
                                    concept_id IN (702953) 
                                    AND is_standard = 0 
                                )) criteria 
                        UNION
                        ALL SELECT
                            criteria.person_id 
                        FROM
                            (SELECT
                                DISTINCT person_id,
                                entry_date,
                                concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                            WHERE
                                (
                                    concept_id IN (706163) 
                                    AND is_standard = 1  
                                    AND  value_as_concept_id IN (9191, 4126681, 45877985, 45884084)
                                )) criteria 
                        UNION
                        ALL SELECT
                            criteria.person_id 
                        FROM
                            (SELECT
                                DISTINCT person_id,
                                entry_date,
                                concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                            WHERE
                                (
                                    concept_id IN (706170) 
                                    AND is_standard = 1  
                                    AND  value_as_concept_id IN (45877985, 45884084)
                                )) criteria 
                        UNION
                        ALL SELECT
                            criteria.person_id 
                        FROM
                            (SELECT
                                DISTINCT person_id,
                                entry_date,
                                concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                            WHERE
                                (
                                    concept_id IN (700360) 
                                    AND is_standard = 1  
                                    AND  value_as_concept_id IN (45884084, 45877985)
                                )) criteria 
                        UNION
                        ALL SELECT
                            criteria.person_id 
                        FROM
                            (SELECT
                                DISTINCT person_id,
                                entry_date,
                                concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                            WHERE
                                (
                                    concept_id IN (706169) 
                                    AND is_standard = 1  
                                    AND  value_as_concept_id IN (45877985, 45884084)
                                )) criteria ) 
                    )
                )"""

dataset_34296773_survey_df = pandas.read_gbq(
    dataset_34296773_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_34296773_survey_df.head(5)

In [ ]:
# This query represents dataset "TESTING_COVID_DATASET" for domain "person" and was generated for All of Us Controlled Tier Dataset v6
dataset_54654031_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (702953) 
                                AND is_standard = 0 
                            )) criteria 
                    UNION
                    ALL SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (706163) 
                                AND is_standard = 1  
                                AND  value_as_concept_id IN (9191, 4126681, 45877985, 45884084)
                            )) criteria 
                    UNION
                    ALL SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (706170) 
                                AND is_standard = 1  
                                AND  value_as_concept_id IN (45877985, 45884084)
                            )) criteria 
                    UNION
                    ALL SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (700360) 
                                AND is_standard = 1  
                                AND  value_as_concept_id IN (45884084, 45877985)
                            )) criteria 
                    UNION
                    ALL SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (706169) 
                                AND is_standard = 1  
                                AND  value_as_concept_id IN (45877985, 45884084)
                            )) criteria ) 
                )"""

dataset_54654031_person_df = pandas.read_gbq(
    dataset_54654031_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_54654031_person_df.head(5)

In [ ]:
# This query represents dataset "TESTING_COVID_DATASET" for domain "survey" and was generated for All of Us Controlled Tier Dataset v6
dataset_54654031_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (
                SELECT
                    DISTINCT(question_concept_id) as concept_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` 
            )
        )  
        AND (
            answer.PERSON_ID IN (
                SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (
                        SELECT
                            criteria.person_id 
                        FROM
                            (SELECT
                                DISTINCT person_id,
                                entry_date,
                                concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                            WHERE
                                (
                                    concept_id IN (702953) 
                                    AND is_standard = 0 
                                )) criteria 
                        UNION
                        ALL SELECT
                            criteria.person_id 
                        FROM
                            (SELECT
                                DISTINCT person_id,
                                entry_date,
                                concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                            WHERE
                                (
                                    concept_id IN (706163) 
                                    AND is_standard = 1  
                                    AND  value_as_concept_id IN (9191, 4126681, 45877985, 45884084)
                                )) criteria 
                        UNION
                        ALL SELECT
                            criteria.person_id 
                        FROM
                            (SELECT
                                DISTINCT person_id,
                                entry_date,
                                concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                            WHERE
                                (
                                    concept_id IN (706170) 
                                    AND is_standard = 1  
                                    AND  value_as_concept_id IN (45877985, 45884084)
                                )) criteria 
                        UNION
                        ALL SELECT
                            criteria.person_id 
                        FROM
                            (SELECT
                                DISTINCT person_id,
                                entry_date,
                                concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                            WHERE
                                (
                                    concept_id IN (700360) 
                                    AND is_standard = 1  
                                    AND  value_as_concept_id IN (45884084, 45877985)
                                )) criteria 
                        UNION
                        ALL SELECT
                            criteria.person_id 
                        FROM
                            (SELECT
                                DISTINCT person_id,
                                entry_date,
                                concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                            WHERE
                                (
                                    concept_id IN (706169) 
                                    AND is_standard = 1  
                                    AND  value_as_concept_id IN (45877985, 45884084)
                                )) criteria ) 
                    )
                )"""

dataset_54654031_survey_df = pandas.read_gbq(
    dataset_54654031_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_54654031_survey_df.head(5)

In [ ]:
dataset_54654031_survey_df = dataset_54654031_survey_df.sort_values('person_id', ignore_index=True)
dataset_54654031_survey_df[dataset_54654031_survey_df['person_id']==1000453]
#This shows that there are about 189 survey questions for each patient.
#I think the best course of action would be to remove irrelavant questions, and quantify the rest
#After that's done, we might be able to find the most important questions by comparing it to long Covid cases

In [ ]:
surveyDf = dataset_54654031_survey_df
surveyDf['survey'].unique()

In [ ]:
#The best thing to do here might be to focus only on the questions that have numerical outputs
intList = range(0,101)
ansList= ['Yes', 'No', 'yes', 'no']
len(surveyDf[surveyDf['answer'].isin(intList)]['question'].unique())
len(surveyDf[surveyDf['answer'].isin(ansList)]['question'].unique())

#It looks like these questions have not been transformed into numeric values yet
#Dictionaries have been created above to help with this however

# Add Long Covid Data and convert survey data

In [ ]:
#storeInBucket('ImportantQuestionsAoU.csv')
importantQuestionDf = getFromBucket('ImportantQuestionsAoU.csv')

In [ ]:
#Lets load our significant questions and long covid status in 
#To load in these files go to -> File -> Open... -> Upload
LongCovidEHRDf = getFromBucket('n3c_aou_cohort.csv')
LongCovidEHRDf

print('Long Covid Size: ' , len(LongCovidEHRDf))            
print('Important Question Size: ' , len(importantQuestionDf))
longCovidDf = LongCovidEHRDf

In [ ]:
#Now what we want to do is filter the questions by the IDs on important question size
importantQuestionDf.head()

In [ ]:
surveyDf = surveyDf[surveyDf[question_concept_id].isin(importantQuestionDf[question_concept_id])]
surveyDf


In [ ]:
#Now we have the surveys filtered, but we need to take those filtered surveys and attach them to our longCovid df
#Each of these question Ids need to be its own column that we will add for each person
longCovidDf = longCovidDf[longCovidDf['person_id'].isin(surveyDf['person_id'])]

#Add a component that takes away the EHR data from this dataframe
longCovidDf = longCovidDf.loc[:, ['person_id', 'y_pred', 'long_covid']]

print(len(longCovidDf))
surveyDf = surveyDf[surveyDf['person_id'].isin(longCovidDf['person_id'])]
print(len(surveyDf))
longCovidDf

In [ ]:
personList = longCovidDf['person_id'].unique()

for question in importantQuestionDf['question'].unique():
    #print(question)
    longCovidDf[question] = str("No Answer")
    longCovidDf= longCovidDf.astype({question : 'str'})
    for person in personList:
        personDf = surveyDf.loc[surveyDf['person_id'] == person]
        try:
            answer = personDf.loc[personDf['question']== question ,'answer'].reset_index(drop=True)[0]
        except KeyError:
            answer = "Error: Not Answered"
        try:
            answer = answer.split(': ')[1]
        except IndexError:
            answer = answer
            #keep the answer the same
        longCovidDf.loc[longCovidDf['person_id'] == person, question] = answer


In [ ]:
longCovidDf= longCovidDf.reset_index(drop=True)
for a in longCovidDf.columns:
    print(a)

# Analyzing Question Types and Making Transformations

In [ ]:
#Make a List of Variables that shows the options that should be voided
obstainOptionsList = ['Skip', "Don't know" , 'Prefer not to answer', 'Prefer Not To Answer','NaN', 'Dont Know',
                      'Sex At Birth None Of These', 'Intersex', 'Not Answered', 'PMI: Skip', 'Response removed due to invalid value'
                      'Sex At Birth None of These', 'Other Arrangement', 'Not Answered', 'No matching concept', 'Response removed due to invalid value']
def checkObstain(answer):
    if (answer in (obstainOptionsList)):
        return np.NaN
    return answer


Binary Variables (Yes-No, USA-Other, Male-Female, Rent-Own )

In [ ]:
#Make List Of Options as well as their matching DictionAries
#If the Dictionaries are Already made from Eric's Work, Copy them here
binaryOptionList = ['Yes', 'No', 'USA', 'Other', 'Rent', 'Own', 'Male', 'Female']
binaryMap = {'Yes': 1, 'No': 0, 'USA':1, 'Other':0, 'Rent':0, 'Own':1, 'Male':0, 'Female':1}
binaryQuestionList = ['Overall Health: Outside Travel 6 Month','The Basics: Birthplace','Hookah Smoking: Hookah Smoke Participant',
                     'Alcohol: Alcohol Participant','Smoking: 100 Cigs Lifetime','Active Duty: Active Duty Serve Status',
                     'Smokeless Tobacco: Smokeless Tobacco Participant','Electronic Smoking: Electric Smoke Participant',
                     'Cigar Smoking: Cigar Smoke Participant','Home Own: Current Home Own','Overall Health: Organ Transplant',
                     'Insurance: Health Insurance','Living Situation: Stable House Concern','Biological Sex At Birth: Sex At Birth']

In [ ]:
for question in binaryQuestionList:
    for i in range(0,len(longCovidDf)):
        answer = longCovidDf.loc[i,question]
        answer = checkObstain(answer)
        if (answer in binaryOptionList):
            longCovidDf.loc[i,question] = binaryMap[answer]
        else:
            longCovidDf.loc[i,question] = answer

Poor-Fair-Good-VeryGood-Excellent

In [ ]:
goodOptionList= ['Poor','Fair','Good', 'Very Good', 'Excellent', 'Excllent' ]
goodMap = {'Poor': 1, 'Fair': 2, 'Good':3,'Very Good':4, 'Excellent':5, 'Excllent':5}
goodQuestionList = ['Overall Health: Social Satisfaction','Overall Health: General Mental Health','Overall Health: General Social',
                   'Overall Health: General Physical Health','Overall Health: General Quality','Overall Health: General Health']

In [ ]:
for question in goodQuestionList:
    for i in range(0,len(longCovidDf)):
        answer = longCovidDf.loc[i,question]
        answer = checkObstain(answer)
        if (answer in goodOptionList):
            longCovidDf.loc[i,question] = goodMap[answer]
        else:
            longCovidDf.loc[i,question] = answer

Often scale

In [ ]:
oftenOptionList = ['Never', 'Rarely', 'Sometimes', 'Often', 'Always', 'Occasionally']
oftenMap = {'Never':0, 'Rarely':1, 'Sometimes':2, 'Often':3,'Occasionally':3, 'Always':4}
oftenQuestionList = ['Overall Health: Emotional Problem 7 Days','Overall Health: Difficult Understand Info','Overall Health: Health Material Assistance']

In [ ]:
for question in oftenQuestionList:
    for i in range(0,len(longCovidDf)):
        answer = longCovidDf.loc[i,question]
        answer = checkObstain(answer)
        if (answer in oftenOptionList):
            longCovidDf.loc[i,question] = oftenMap[answer]
        else:
            longCovidDf.loc[i,question] = answer

Severe Scale

In [ ]:
severeOptionList = ['None', 'Mild', 'Moderate', 'Severe', 'Very Severe']
severeMap = {'None':0, 'Mild':1, 'Moderate':2, 'Severe':3, 'Very Severe':4}
severeQuestionList = ['Overall Health: Average Fatigue 7 Days']

In [ ]:
for question in severeQuestionList:
    for i in range(0,len(longCovidDf)):
        answer = longCovidDf.loc[i,question]
        answer = checkObstain(answer)
        if (answer in severeOptionList):
            longCovidDf.loc[i,question] = severeMap[answer]
        else:
            longCovidDf.loc[i,question] = answer

How Much it Describes You

In [ ]:
describeOptionList = ['Not At All', 'A Little Bit', 'Somewhat', 'Quite a Bit','Quite A Bit', 'Extremely']
describeMap = {'Not At All':0, 'A Little Bit':1, 'Somewhat':2, 'Quite a Bit':3,'Quite A Bit':3,  'Extremely':4}
describeQuestionList = ['Overall Health: Medical Form Confidence']

In [ ]:
for question in describeQuestionList:
    for i in range(0,len(longCovidDf)):
        answer = longCovidDf.loc[i,question]
        answer = checkObstain(answer)
        if (answer in describeOptionList):
            longCovidDf.loc[i,question] = describeMap[answer]
        else:
            longCovidDf.loc[i,question] = answer

In [ ]:
describe2OptionList = ['Not At All', 'A Little', 'Moderately', 'Mostly', 'Completely', 'Quite A Bit']
describe2Map = {'Not At All':0, 'A Little':1, 'Moderately':2, 'Mostly':3,'Quite A Bit':3,  'Completely':4}
describe2QuestionList = ['Overall Health: Everyday Activities']

In [ ]:
for question in describe2QuestionList:
    for i in range(0,len(longCovidDf)):
        answer = longCovidDf.loc[i,question]
        answer = checkObstain(answer)
        if (answer in describe2OptionList):
            longCovidDf.loc[i,question] = describe2Map[answer]
        else:
            longCovidDf.loc[i,question] = answer

Already Scalar

In [ ]:
scalarOptionsList = range(0,11)
scalarQuestionList = ['Living Situation: People Under 18', 'Living Situation: How Many People','Overall Health: Average Pain 7 Days']

In [ ]:
for question in scalarQuestionList:
    for i in range(0,len(longCovidDf)):
        answer = longCovidDf.loc[i,question]
        answer = checkObstain(answer)
        if longCovidDf.loc[i,question] == '6 or more household members under the age of 18':
            answer = 6
        try:
            longCovidDf.loc[i,question] = int(answer)
            
        except ValueError:
            longCovidDf.loc[i,question] = np.NaN
        

Other

In [ ]:
#Question: Race
#Answers:White, Black, Hispanic, Asian, MENA, NHPI, None of These, Prefer not to answer, Skip
#change to 3 Variables: white, black, or other race. Each its own binary variable
otherList = []

#First, make a white Yes, No
question = 'Race: What Race Ethnicity'
otherList = otherList +[question]
for i in range(0,len(longCovidDf)):
    answer = longCovidDf.loc[i,question]
    answer = checkObstain(answer)
    if (answer == 'White'):
        longCovidDf.loc[i,question] = 1
    else:
        longCovidDf.loc[i,question] = 0

In [ ]:
#Question:Recreation Drug Use: Which Drugs Used
#Answers: Cocain, Prescription Opiods, Marijuana, Prefer Not Answer, None of These Drugs, 
    #Inhalants, Hallucinogens, Other, Sedatives, Prescription Stimulants, Methamphetamine, Street Opiods, Skip
#Make this 2 binary options: Prescription drugs (y/n) and Non-perscription drugs(y/n)

#First, Make a simple Yes/No for all Drugs

question = 'Recreational Drug Use: Which Drugs Used'
otherList = otherList +[question]
for i in range(0,len(longCovidDf)):
    answer = longCovidDf.loc[i,question]
    answer = checkObstain(answer)
    if (answer == 'None of These Drugs'):
        longCovidDf.loc[i,question] = 0
    else:
        longCovidDf.loc[i,question] = 1

In [ ]:
#Question:Sexual Orientation
#Answers:Straight, Bisexual, Gay, Lesbian, None, Skip, Prefer Not TO Answer

#Make different variables for LGBTQ+ and straight
#Start with straight Y/N

question = 'The Basics: Sexual Orientation'
otherList = otherList +[question]
for i in range(0,len(longCovidDf)):
    answer = longCovidDf.loc[i,question]
    answer = checkObstain(answer)
    if (answer == 'Straight'):
        longCovidDf.loc[i,question] = 1
    else:
        longCovidDf.loc[i,question] = 0

In [ ]:
#Question:Annual Income
#Answers:Less than 10K, 10K - 25K, 25K-35K, 35K-50K, 50K-75K, 75K-100K, 100K-150K, 150K-200K, More than 200K, 
    #Prefer not to answer, Skip, 
    
#Make a 1-9 scalar

question = 'Income: Annual Income'
otherList = otherList+[question]

incomeOptionList = ['less 10k', '75k 100k',
 '35k 50k', 'more 200k',
 '10k 25k', '100k 150k',
 '25k 35k', '50k 75k',
 '150k 200k']

incomeMap = {'less 10k':0, '10k 25k':1, '25k 35k': 2, '35k 50k':3, '50k 75k':4, '75k 100k':5, '100k 150k':6, '150k 200k':7, 'more 200k':8}

for i in range(0,len(longCovidDf)):
        answer = longCovidDf.loc[i,question]
        answer = checkObstain(answer)
        if (answer in incomeOptionList):
            longCovidDf.loc[i,question] = incomeMap[answer]
        else:
            longCovidDf.loc[i,question] = answer

In [ ]:
#Question:Current Marital Status
#Answers:Never Married, Married, Living With Partner, Divorced, Seperated, Widowed, Skip, Prefer Not Answer
#Make 2 binary variables "Ever Been Married" and "Currently married"

#Start with "have you been married"
question = 'Marital Status: Current Marital Status'
otherList = otherList+[question]

marriageOptionList=[ 'Never Married', 'Married',
 'Living With Partner',
 'Divorced', 'Separated',
 'Widowed']

marriageMap = {'Never Married':0, 'Married':1, 'Living With Partner': 0, 'Divorced':1, 'Widowed':1, 'Separated':1}

for i in range(0,len(longCovidDf)):
        answer = longCovidDf.loc[i,question]
        answer = checkObstain(answer)
        if (answer in marriageOptionList):
            longCovidDf.loc[i,question] = marriageMap[answer]
        else:
            longCovidDf.loc[i,question] = answer

In [ ]:
#Question:Alchohol: Drink Frequency Past Year
#Answers: Never, Monthly or Less, 2 to 4 per month, 2 to 3 per week, 4 or more per week, Prefer Not to Answer, Skip
#Make 0-4 scalar

question = 'Alcohol: Drink Frequency Past Year'
otherList = otherList+[question]

alcoholOptionList = ['Monthly Or Less',
 '4 or More Per Week',
 '2 to 4 Per Month',
 '2 to 3 Per Week',
 'Never']

alcoholMap = {'Never':0, 'Monthly Or Less':1, '2 to 4 Per Month': 2, '2 to 3 Per Week':3, '4 or More Per Week':4}

for i in range(0,len(longCovidDf)):
        answer = longCovidDf.loc[i,question]
        answer = checkObstain(answer)
        if (answer in alcoholOptionList):
            longCovidDf.loc[i,question] = alcoholMap[answer]
        else:
            longCovidDf.loc[i,question] = answer

In [ ]:
#Question: Employment Status
#Answers: Out of Work/Less than One, Employed for Wages, Retired, Unable to Work, Student, Self Employed
#Out of Work /One or More, Homemaker, Skip, Prefer Not Answer\
#Make Binary Variable "Working or not"

question = 'Employment: Employment Status'
otherList = otherList +[question]
workingList = ['Self Employed', 'Employed for Wages']

for i in range(0,len(longCovidDf)):
    answer = longCovidDf.loc[i,question]
    answer = checkObstain(answer)
    if (answer in workingList):
        longCovidDf.loc[i,question] = 1
    else:
        longCovidDf.loc[i,question] = 0

In [ ]:
#Question:Living Situation: How Many YEars
#Answers:less than 1, 1 to 2, 3 to 5, 6 to 10, 11 to 20, more than 20, skip
#Make a 0-5 scalar

question = 'Living Situation: How Many Living Years'
otherList = otherList+[question]

livingOptionList = ['less 1', '3 to 5',
 '11 to 20', 'more 20',
 '6 to 10', '1 to 2']

livingMap = {'less 1':0, '1 to 2':1, '3 to 5': 2, '6 to 10':3, '11 to 20':4, 'more 20':5}

for i in range(0,len(longCovidDf)):
        answer = longCovidDf.loc[i,question]
        answer = checkObstain(answer)
        if (answer in livingOptionList):
            longCovidDf.loc[i,question] = livingMap[answer]
        else:
            longCovidDf.loc[i,question] = answer

In [ ]:
#Question:Education Level: Highest Grade
#Answers:Twelve or GED, Advanced Degree, College One to Three, Five Through Eight, Nine Throigh Eleven, College Graduate, 
#Prefer Not Answer, Skip, Never Attended, One through Four

#Use Scalar

question = 'Education Level: Highest Grade'
otherList = otherList+[question]

eduOptionList = ['Twelve Or GED',
                 'Advanced Degree', 'College One to Three', 'Five Through Eight', 'Nine Through Eleven',
                 'College Graduate', 'Never Attended','One Through Four']

eduMap = education7map

for i in range(0,len(longCovidDf)):
        answer = longCovidDf.loc[i,question]
        answer = checkObstain(answer)
        if (answer in eduOptionList):
            longCovidDf.loc[i,question] = eduMap[answer]
        else:
            longCovidDf.loc[i,question] = answer

In [ ]:
# Now we need to take each of these questions and make an answer for them that is some form of scalar. This may Take Some time.
#A good way might be to make a loop given the "important questions" data frame to look at the unique entries of each
for question in surveyDf['question'].unique():
    if(question in ['Overall Health: Average Pain 7 Days']):
        print(question)
        thisQuestionDf = surveyDf[surveyDf['question']==question]
        print(thisQuestionDf['answer'].unique())

In [ ]:
numericList = binaryQuestionList+describeQuestionList+describe2QuestionList+severeQuestionList+scalarQuestionList+oftenQuestionList+goodQuestionList+['long_covid', 'person_id']+otherList
#numericList

In [ ]:
trialDf=longCovidDf

In [ ]:
nanList = []
for a in range(0,len(trialDf)):
    if (trialDf.loc[a,'y_pred'] >=0):
        #do nothing
        x=1
    else:
        nanList = nanList + [a]
        print(a, trialDf.loc[a,'y_pred'])

In [ ]:
for a in nanList:
    trialDf.loc[a,'y_pred'] = 0.40883501377605025

In [ ]:
#There are targetVariables and y_pred variables that are 'too large to be float 64. Try to find them and see what it is
for i in range(0,len(trialDf)):
    if (float(trialDf.loc[i,'y_pred'])<= 1):
        x=1
    else:
        print(trialDf.loc[i,'y_pred'])

## Find where the code is giving non-numeric outputs

In [ ]:
#print Unique values for some of the columns. If nothing turns up, then make an iterative try-catch loop to find non-numerics
for a in trialDf.columns:
    if type(trialDf.loc[1,a]) in [float, int, np.int64, np.float64]:
        x=1
    else:
        print (a)
        print(trialDf[a].unique())

In [ ]:
trialDf = trialDf.drop(['Health Insurance: Health Insurance Type', 'Health Insurance: Insurance Type Update', 'Gender: Gender Identity'], axis=1)

In [ ]:
##Make a big loop to check all the answers, and hope it doesn't take forever
for col in trialDf.columns:
    for row in range(0,len(trialDf)):
        trialDf.loc[row,col] = checkObstain(trialDf.loc[row,col])
        

In [ ]:
trialDf = trialDf.replace('NaN', np.NaN)
trialDf = trialDf.replace('nan', np.NaN)
for a in trialDf.columns:
    #columnDf = trialDf[a]
    columnMean = trialDf.loc[trialDf[a].isin(range(0,100)),a].mean()
    #print(a)
    #print(columnMean)
    trialDf[a] = trialDf[a].fillna(value = columnMean)


# Export Df using google bucket

In [ ]:
#trialDf=trialDf.drop(['index'] ,axis=1)
trialDf

In [ ]:
my_dataframe = trialDf   

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'SurveyDataJuly2023.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr


In [ ]:
#.........

In [ ]:
#-----